In [42]:
import pandas as pd
import math
import numpy as np

In [43]:
""" 
Append the Branch information to the Customer information
Transform the latitude and longitude into radians
Find the closest Branch for each Customer
Make sure Distance is rounded to 2 decimal places
For each Branch, assign a Customer Priority rating, the closest customer having a rating of 1
"""

' \nAppend the Branch information to the Customer information\nTransform the latitude and longitude into radians\nFind the closest Branch for each Customer\nMake sure Distance is rounded to 2 decimal places\nFor each Branch, assign a Customer Priority rating, the closest customer having a rating of 1\n'

In [44]:
#Input data
branches = pd.read_csv('DSB Branches.csv')
customers = pd.read_csv("DSB Customer Locations.csv")

In [45]:
#Create dummy field to union both dataframes together

customers['Dummy field'] = 1
branches['Dummy field'] = 1
df = pd.merge(branches, right=customers, on='Dummy field' )
df.drop(['Dummy field'], inplace=True, axis = 1)

In [46]:
#Turn Lat and Long into radians
rad_converter = 180/math.pi

df['Branch Long - radians'] = df['Branch Long'] / rad_converter
df['Branch Lat - radians'] = df['Branch Lat'] / rad_converter

df['Address Long - radians'] = df['Address Long'] / rad_converter
df['Address Lat - radians'] = df['Address Lat'] / rad_converter


In [47]:
df

,Branch,Branch Long,Branch Lat,Customer,Address Long,Address Lat,Branch Long - radians,Branch Lat - radians,Address Long - radians,Address Lat - radians
0,London,-0.109863,51.481383,1,0.747070,51.549751,-0.001917,0.89852,0.013039,0.899713
1,London,-0.109863,51.481383,2,0.406494,51.957807,-0.001917,0.89852,0.007095,0.906835
2,London,-0.109863,51.481383,3,-0.142822,50.833698,-0.001917,0.89852,-0.002493,0.887215
3,London,-0.109863,51.481383,4,-4.640350,50.335819,-0.001917,0.89852,-0.080989,0.878526
4,London,-0.109863,51.481383,5,-1.593018,51.289406,-0.001917,0.89852,-0.027803,0.895169
...,...,...,...,...,...,...,...,...,...,...
495,Newcastle,-1.604004,54.965002,121,-0.063171,51.559997,-0.027995,0.95932,-0.001103,0.899892
496,Newcastle,-1.604004,54.965002,122,-0.263672,51.623132,-0.027995,0.95932,-0.004602,0.900994
497,Newcastle,-1.604004,54.965002,123,-0.299377,51.390637,-0.027995,0.95932,-0.005225,0.896936
498,Newcastle,-1.604004,54.965002,124,-0.304871,51.376924,-0.027995,0.95932,-0.005321,0.896696


In [48]:
#Haversine formula: 3963 * acos((sin(lat1) * sin(lat2)) + cos(lat1) * cos(lat2) * cos(long2 – long1))

def calculate_distance(row):
    lat1 = row['Branch Lat - radians']
    lon1 = row['Branch Long - radians']
    lat2 = row['Address Lat - radians']
    lon2 = row['Address Long - radians']

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (math.sin(lat1) * math.sin(lat2)) + + math.cos(lat1) * math.cos(lat2) * math.cos(lon2 - lon1)
    c = math.acos(a)
    distance = round(3963 * c,2)  # Radius of the Earth in miles

    return distance

df['Distance (miles)'] = df.apply(calculate_distance, axis=1)

In [49]:
#Rank the branches for each customer
df['Rank'] = df.groupby('Customer')['Distance (miles)'].rank(ascending=True)

In [50]:
#Filter the top branch for each customer and drop unneeded columns
df.sort_values(['Customer','Rank'])
df = df[df['Rank'] == 1]
df.drop(['Branch Long - radians', 'Branch Lat - radians', 'Address Long - radians' ,'Address Lat - radians','Rank'],axis = 1, inplace= True)

In [52]:
#Create a customer priority for each branch using the closest customers only
df['Customer Priority'] = df.groupby('Branch')['Distance (miles)'].rank(ascending=True)

C:\Users\Dell\AppData\Local\Temp\ipykernel_9732\4213867834.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Customer Priority'] = df.groupby('Branch')['Distance (miles)'].rank(ascending=True)


In [53]:
df.to_csv('output.csv')

In [54]:
df

,Branch,Branch Long,Branch Lat,Customer,Address Long,Address Lat,Distance (miles),Rank,Customer Priority
0,London,-0.109863,51.481383,1,0.747070,51.549751,37.19,1.0,40.0
1,London,-0.109863,51.481383,2,0.406494,51.957807,39.69,1.0,42.0
2,London,-0.109863,51.481383,3,-0.142822,50.833698,44.82,1.0,43.0
7,London,-0.109863,51.481383,8,1.296387,52.616390,98.69,1.0,50.0
25,London,-0.109863,51.481383,26,1.494141,52.308479,89.22,1.0,49.0
...,...,...,...,...,...,...,...,...,...
408,Newcastle,-1.604004,54.965002,34,-3.208008,54.393352,75.35,1.0,15.0
409,Newcastle,-1.604004,54.965002,35,-2.526855,54.826008,37.95,1.0,6.0
411,Newcastle,-1.604004,54.965002,37,-1.867676,53.917281,73.24,1.0,14.0
465,Newcastle,-1.604004,54.965002,91,-1.406250,54.226708,51.68,1.0,8.0
